In [1]:
# Import all the necessary libraries for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import datetime
# plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# For the machine learning model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [2]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install plotly
# !pip install scikit-learn

In [3]:
# # get out the current directory
# current_dir = os.getcwd()
# # get out the parent directory
# parent_dir = os.path.dirname(cuuser_dfrrent_dir)
# # change to the parent directory
# os.chdir(parent_dir)

In [24]:
path = "../inter-uni-datathon-2024-nsw/"
train_df = pd.read_csv(path + "train.csv")
test_df = pd.read_csv(path + "test.csv")

In [25]:
# Age (does young adult cause more fraud?)
user_info = [
    "Age", 
    "Terrorism",
    "Income",
    "MaritalStatus",
    "Occupation",
    "EducationLevel",
    "NumDependents",
    "GiftsTransaction" 
]
transaction_info = [
    "TransactionType", 
    "TransactionDate",
    "TransactionTime", 
    "TransactionAmount",
    "TransactionLocation" 
]
account_info = [
    "UserTenure",
    "Expenditure",
    "Latitude",
    "Longitude" 
]

In [29]:
train_df

,TransactionNumber,UserID,Age,Gender,Occupation,EducationLevel,MaritalStatus,NumDependents,Income,Expenditure,...,UserTenure,IsFraud,Income_Value,Income_Currency,Expenditure_Value,Expenditure_Currency,GiftsTransaction_Value,GiftsTransaction_Currency,TransactionAmount_Value,TransactionAmount_Currency
0,8765,70,37,Female,Professional,Bachelor,Widowed,3,28884.43 AUD,14610.61 AUD,...,113,1,28884.43,AUD,14610.61,AUD,1050.01,AUD,258.14,AUD
1,9645,3386,34,Male,Student,High School,Married,4,AU$ 54919.07,39169.49 AUD,...,104,1,54919.07,AUD,39169.49,AUD,4969.71,AUD,34.94,AUD
2,1145,2971,25,Male,Unemployed,Master,Married,2,AU$ 74728.57,55873.76 AUD,...,105,0,74728.57,AUD,55873.76,AUD,1149.85,AUD,323.82,AUD
3,15308,2925,25,Male,Professional,High School,Married,3,AU$ 55712.62,AED 89649.04,...,70,1,55712.62,AUD,89649.04,AUD,4335.70,AUD,32.49,AUD
4,14967,2339,38,Male,Professional,High School,Single,4,53004.7 AUD,AED 43601.02,...,27,0,53004.70,AUD,43601.02,AUD,4763.48,AUD,1140.75,AUD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10961,11284,3632,25,Male,Unemployed,High School,Single,3,64488.68 AUD,AU$ 21813.53,...,110,0,64488.68,AUD,21813.53,AUD,2744.53,AUD,182.51,AUD
10962,11964,3925,-18,Female,Professional,High School,Married,2,80403.31 AUD,AU$ 63429.08,...,91,1,80403.31,AUD,63429.08,AUD,191.21,AUD,137.50,AUD
10963,5390,4811,22,Male,Unemployed,High School,Widowed,3,AU$ 29048.42,18806.31 AUD,...,84,0,29048.42,AUD,18806.31,AUD,51.03,AUD,112.37,AUD
10964,860,1110,29,Female,Student,High School,Divorced,4,28654.66 AUD,AU$ 9748.53,...,62,1,28654.66,AUD,9748.53,AUD,1179.41,AUD,68.05,AUD


## EDA

### Clean currency data (4 columns)

In [27]:
import re
import pandas as pd

def get_currency_and_symbol(val):
    if 'AUD' or 'AU' in val:
        # create a feature for aud
        return re.sub(r'[^\d.]', '', val), 'AUD' 
    elif 'AED' in val:
        value = re.sub(r'[^\d.]', '', val)
        # convert AED to AUD
        return float(value) * 0.35, 'AED'
    elif '£' in val:
        value = re.sub(r'[^\d.]', '', val)
        # convert pounds to AUD
        return float(value) * 1.8, 'GBP'
    elif 'GBP' in val:
        value = re.sub(r'[^\d.]', '', val)
        # convert pounds to AUD
        return float(value) * 1.8, 'GBP'
    else:
        return "Unidentified", "Unknown" 


# Apply to Income
train_df['Income'], train_df['Income_Currency'] = zip(*train_df['Income'].apply(get_currency_and_symbol))
train_df['Income'] = pd.to_numeric(train_df['Income'], errors='coerce')  

# Apply to Expenditure
train_df['Expenditure'], train_df['Expenditure_Currency'] = zip(*train_df['Expenditure'].apply(get_currency_and_symbol))
train_df['Expenditure'] = pd.to_numeric(train_df['Expenditure'], errors='coerce')

# Apply to GiftsTransaction
train_df['GiftsTransaction'], train_df['GiftsTransaction_Currency'] = zip(*train_df['GiftsTransaction'].apply(get_currency_and_symbol))
train_df['GiftsTransaction'] = pd.to_numeric(train_df['GiftsTransaction'], errors='coerce')

# Apply to TransactionAmount
train_df['TransactionAmount'], train_df['TransactionAmount_Currency'] = zip(*train_df['TransactionAmount'].apply(get_currency_and_symbol))
train_df['TransactionAmount'] = pd.to_numeric(train_df['TransactionAmount'], errors='coerce') 


In [8]:
# Check if there's any missing values
train_df['Income'].isnull().sum()
train_df[train_df['Income'] == 'Unidentified']
# expenditure
train_df['Expenditure'].isnull().sum()
train_df[train_df['Expenditure'] == 'Unidentified']
# giftsTransaction
train_df['GiftsTransaction'].isnull().sum()
train_df[train_df['GiftsTransaction'] == 'Unidentified']
# TransactionAmount
train_df['TransactionAmount'].isnull().sum()
train_df[train_df['TransactionAmount'] == 'Unidentified']

,TransactionNumber,UserID,Age,Gender,Occupation,EducationLevel,MaritalStatus,NumDependents,Income,Expenditure,...,MerchantID,TransactionType,TransactionLocation,DeviceType,Latitude,Longitude,EmailDomain,Terrorism,UserTenure,IsFraud


In [9]:
# Convert categorical data to numerical data through One Hot Encoding
user_df = pd.get_dummies(train_df[user_info])
transaction_df = pd.get_dummies(train_df[transaction_info])
account_df = pd.get_dummies(train_df[account_info])
# Append IsFraud to the all dfs
user_df['IsFraud'] = train_df['IsFraud']
transaction_df['IsFraud'] = train_df['IsFraud']
account_df['IsFraud'] = train_df['IsFraud']

In [15]:
# Calculate correlations once for each DataFrame
user_corr = user_df.corrwith(user_df['IsFraud'])
transaction_corr = transaction_df.corrwith(transaction_df['IsFraud'])
account_corr = account_df.corrwith(account_df['IsFraud'])

# Sort the correlations in descending order
user_corr = user_corr.sort_values(ascending=False)
transaction_corr = transaction_corr.sort_values(ascending=False)
account_corr = account_corr.sort_values(ascending=False)


In [20]:
user_corr.head(10)

IsFraud                    1.000000
MaritalStatus_Divorced     0.032624
EducationLevel_PhD         0.020876
Occupation_Unemployed      0.015259
GiftsTransaction           0.014263
Terrorism                  0.013633
Income                     0.012592
MaritalStatus_Widowed      0.008158
Occupation_Student         0.007891
EducationLevel_Bachelor   -0.001135
dtype: float64

In [19]:
transaction_corr.head(10)

IsFraud                         1.000000
TransactionType_Withdrawal      0.557298
TransactionTime_00:00:00        0.153296
TransactionAmount               0.103385
TransactionTime_00/00/00        0.054156
TransactionTime_12:00:00 AM     0.050333
TransactionLocation_Brisbane    0.043807
TransactionLocation_Perth       0.039812
TransactionDate_2023-05-26      0.034838
TransactionLocation_Hobart      0.033201
dtype: float64

In [18]:
account_corr

IsFraud        1.000000
Expenditure    0.011879
Latitude       0.005265
UserTenure     0.004936
Longitude      0.003647
dtype: float64

In [23]:
# export train_df to csv
train_df.to_csv('train_cleaned.csv', index=False)